# 大きいサイズにチャレンジ
分割して処理し，統合してみる

# Great Fake Painter

Fake Painter はどんなサイズの画像でも224x224にリサイズして処理していましたが，もっと大きな絵を描かせてみたいな，という要望に応えてみました．

動作環境
- Windows, MacOS, Linux で動作確認済み。いずれも Anaconda
- chainer 2.0
- python3.6 （3.5でもよい）
- opencv 3.2 (3.0でもよい。２でもよいかも） PILだけでも書けると思うが、本人が PIL より慣れてるためちょっとだけ使った。 

縦も横も200の倍数＋224でもっとも画像サイズに近いサイズに画像をリサイズし，224×224のブロック単位で処理します．
単純に分割処理すると境界がくっきりと目立ってしまって連続性がなくなるので，隣のブロックとは24ピクセル分重ねることにしました．

In [ ]:
import chainer.functions as F
import chainer
from chainer.links import VGG16Layers
from chainer.links.model.vision.vgg import prepare as VGGprepare
from chainer import Variable, optimizers

import numpy as np
import PIL.Image
from PIL import ImageFilter
from io import BytesIO
import urllib.request
import cv2
import sys

#from IPython.html.widgets import FloatProgress
from ipywidgets import FloatProgress
from IPython.display import clear_output, Image, display

mean = [103.939, 116.779, 123.68]   # BGR

In [ ]:
# 基本入出力関数

# url を指定して画像を読み込み
def url2img(url):
    url_response = urllib.request.urlopen(url)
    img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, -1)
    img =PIL.Image.fromarray(img[:, :, ::-1].copy())
    return img

# ファイルまたはウェブ上のファイルを読み込み
def getImage(filename):
        if filename[:4] == "http":
            img = url2img(filename)
        else:
            img = PIL.Image.open(filename)
        return img

# numpy画像の表示
def showarray(a, fmt='jpeg'):  # 
    a = np.uint8(np.clip(a, 0, 255))
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

# blob データを PIL 画像に変換
def blob2img(blob, mean=mean):
    blob = (np.dstack(blob)+ mean)[:,:,::-1]   # BGR 2 RGB
    return PIL.Image.fromarray(np.uint8(np.clip(blob,0,255)))

# blob データを画像として保存
def save_image(blobimg, it, fn = "frames",mean =mean):
    image = blob2img(blobimg, mean=mean)
    image.save(fn+"/im_%05d.png"%it)
    return image

# チャネル間の相関行列
def ch_corr_matrix(ld):
    # 0次元目はバッチ数
    ch = ld.shape[1] # チャネル数
    size = ld[0][0].size  # チャネルあたりのデータ数
    cmatrix = F.reshape(ld, (ch,size)) # チャネルごとに１次元化した ch x size の配列を作る
    matrix = F.matmul(cmatrix, cmatrix,transb=True) / np.float32(ch*size) # 相関行列
    return matrix

In [ ]:
# layers_for_orig = ["conv4_1", "conv4_2", "conv4_3", "pool4"]
# layers_for_style =["conv1_1", "conv1_2", "pool1", "conv2_1", "conv2_2", "pool2", "conv3_1", "conv3_2", "conv3_3v3","pool3"]

CROPSIZE = (3,224,224)

# Fake Painter -- VGG16Layer の拡張クラス
class FakePainter(VGG16Layers):
    def __init__(self):
        super(FakePainter, self).__init__()
        with self.init_scope():
            self.autopic = chainer.links.Parameter(self.genpic())
            self.opic = np.zeros(CROPSIZE ,dtype='f')
            self.spic = np.zeros(CROPSIZE ,dtype='f')
    
    def __call__(self, vimg, layers=['prob']):
         return self.myextract(vimg, layers=layers)
            
    def myextract(self,vimg, layers=['prob']):
        if vimg.data.ndim == 3:
            vimg = F.reshape(vimg,(1,3,224,224))
        return super(FakePainter,self).__call__(vimg,layers=layers)
    
    def genpic(self):
        genpic = np.random.uniform(-20,20,CROPSIZE ).astype(np.float32)
        return genpic
    
    def setpic(self, nimg):
        self.autopic.W.data = nimg
 
    def show(self,mode=0):
        if mode == 1 :
           img = blob2img(self.opic)
        elif mode == 2 :
           img = blob2img(self.spic)
        else :
           img = blob2img(self.autopic.data)  
        return img
        
    # VGGprepare を使って画像をVGG16用に変換
    def setImage(self,img, withsmooth=False):
        nimg = VGGprepare(img)
        if withsmooth:
            mag = min(img.size)/224
            self.smimg = img.filter(ImageFilter.GaussianBlur(3*mag))# ぼかしたイメージ 
            nsmimg = VGGprepare(self.smimg)
            return nimg, nsmimg
        else:
            return nimg
        
    # autopic をもとに前向き伝搬
    def autoforward(self,layers=['prob']):
        h = self.myextract(self.autopic, layers=layers)
        return h
    
    # 処理対象画像をセット
    def setOimage(self,img, layers=['vpool4']):
        self.opic, smpic =  self.setImage(img, withsmooth=True)
        vimg = Variable(self.opic)
        h = self.myextract(vimg,layers=layers)  # 対象画像の特徴情報
        self.Ofeatures = h
        self.vsmimg = smpic
        
    # スタイル画像をセット
    def setSimage(self,img,layers=['conv4_1']):
        self.spic =  self.setImage(img)
        vimg = Variable(self.spic)
        h = self.myextract(vimg,layers=layers)
        cor = {}
        for l in h.keys():
            cor.update({l:ch_corr_matrix(h[l])})
        self.Sfeatures = h
        self.Scorr = cor

################ 
nn=FakePainter()    ##  <- これが主役
################

In [ ]:
# 基本となる 224x224 サイズの画像の描画
def generate224(nn,oimg,styleimg,itr,l1,l2,dlosslimit):
        global progress2        
        nn.setOimage(oimg,l1)
        nn.setSimage(styleimg,l2)
        
        # パラメータ
        oldloss = np.array([1e+30,1e+30,1e+30,1e+30]) # 
        tolerance = 2 # この回数以上 loss が前より上昇したらあきらめる
        upcount = 0 # loss が連続上昇した回数のカウンタ

        # train mode
        chainer.config.train=False
        chainer.config.enable_backprop=True

        optimizer = optimizers.Adam(alpha=lr, beta1=0.5)
        optimizer.setup(nn.autopic)

        # oldx = nn.autopic.W  # 　前回の　x の値の初期値     myoptimize を使う場合はこの2行のコメントをはずす
        # oldx.grad = np.zeros_like(oldx.data)  # grad はゼロにセット

        for times  in range (itr):
            nn.cleargrads()  # 微係数データを初期化                
            loss = Variable(np.zeros(()).astype(np.float32))
            loss1 = Variable(np.zeros(()).astype(np.float32))  # 入力画像との自乗誤差
            loss2 = Variable(np.zeros(()).astype(np.float32))  # スタイル画像とのチャネル相関の自乗誤差

            #  l1での現画像と入力画像の誤差を求める
            x = nn.autopic.W
            # 前向き伝搬して指定した層のデータを取得
            efs = nn.myextract(x,layers=list(set(l1+l2)))
            
            for key in l1:
                ef = efs[key]
                of = nn.Ofeatures[key].data  # 先に保存してある対象画像の指定層データ
                loss1 += F.mean_squared_error(ef,of)   # 距離を誤差とする
 
            #  l2での現画像とスタイル画像のチャネル相関誤差を求める
            for key in l2:
                ef = efs[key]
                of = Variable(nn.Sfeatures[key].data) # 先に保存してあるスタイル画像の指定データ
                ecm = ch_corr_matrix(ef)   # チャネル相関マトリクス
                ocm = nn.Scorr[key].data  # 先に保存してあるスタイル画像のチャネル相関マトリクス
                loss2 += F.mean_squared_error(ecm, ocm)  # マトリクス間の自乗誤差

            loss3 = F.mean_squared_error(x, Variable(nn.vsmimg))    # 元の画像におおまかに近づけるための項
           
            loss = rr1*loss1+rr2*loss2+(1-rr1-rr2)*loss3
                        
            loss.backward()

            xgrad = x.grad   #　 loss に対する入力の寄与を保存
            nn.cleargrads()  # nn 全体の微分データをクリア
            x.grad = xgrad

            # chainer のオプティマイザーを使わない場合はこの4行のコメントを外す
            # def myoptimize(oldvimg, vimg, alpha=0.1, beta=0.5):
            #    vimg.data -= alpha * ( vimg.grad + beta * oldvimg.grad)
            # return vimg
            # x = myoptimize(oldx, x, alpha = lr, beta = 0.5)

            # chainer のオプティマイザー
            optimizer.update()
           
            nn.autopic.W = x   # 自動生成画像を更新データで上書き
            
            # 0+xxx としているのは，そうしないと formet が型エラーを吐くから
            tl, tl1,tl2,tl3 = 0+loss.data, 0+rr1*loss1.data, 0+rr2*loss2.data, 0+(1-rr1-rr2)*loss3.data
            if times > 0:
                sys.stdout.write("\r{:5d} {:.5f}  = {:.5f} +{:.5f}+ {:.5f}  ( {:.5f}  {:.5f}  {:.5f} ({}))".format(times, tl, tl1, tl2, tl3, tl1-oldloss[1], tl2-oldloss[2], tl3-oldloss[3],upcount))
                sys.stdout.flush()
                
            progress2.value = times
            
            # oldx = x    #  1回前の x を記憶  myoptimize を使う場合はコメントを外す
            
            if   loss.data < dlosslimit:
                print("評価基準を超えた")
                break
            if   loss.data > oldloss[0]:
                upcount = upcount + 1
            else:
                upcount = 0
            if upcount > tolerance:
                print("ロスが増えつづけているので停止")
                break
                
            oldloss = (tl, tl1,tl2,tl3)   
 
        return blob2img(nn.autopic.W.data,mean=mean),loss.data
            


In [ ]:
# 描画メソッド
def generate3(nn,oimg,style, steps=1,itr=100,cont=False,l1=['conv4_3','pool4'], \
              l2=['conv4_1','conv4_2','conv4_3','pool4'],\
             fn="frames", dlosslimit = 100):
    # nn: ネットワーク、oimg：対象画像名, style：スタイル画像, cont: 前回の続き、\
    # itr:繰り返し回数, l1: oimgから取り出す階層、l2:style から取り出す階層, 
    # fn: 出力フォルダ名, 
    # dlossliit: loss がこれ以上下がったら終わる 

    # 描画対象画像の読み込み
    Oimg = PIL.Image.open(oimg)  # 原画像
    (width,height) = Oimg.size   # 原画像のサイズ
    # スタイル画像をセット
    styleimg = getImage(style)

    # Progress bar
    global progress0,progress1,progress2
    progress0 = FloatProgress(min=0, max=steps-1)
    progress1 = FloatProgress(min=0, max=(int((width-223)/200)+1)*(int((height-223)/200)+1))    
    progress2 = FloatProgress(min=0, max=itr-1)
    display(progress0)
    display(progress1)
    display(progress2)

    bimg = Oimg.copy().filter(ImageFilter.GaussianBlur(25))
    nn.autopic.W.data = VGGprepare(bimg) # autopic にぼかした画像をセット
    
    # 画像全体を使って１サイクル描画を実行する．これを拡大分割したものを各ブロックの種とする
    Firstimg,_= generate224(nn,Oimg,styleimg,itr=itr,l1=l1, l2=l2, \
                                  dlosslimit=dlosslimit)
     
    # 200の倍数＋224で画像より小さくかつもっとも画像サイズに近いサイズを計算
    width1 = (int((width-224)/200))*200+224 
    height1 = (int((height-224)/200))*200+224  
    Oimg = Oimg.resize(np.array((width1,height1))) # 原画像のサイズをフィットさせる
    # ぼかしのために枠をつける
    Bimg = PIL.Image.new('RGB', (width1+50, height1+50), (200, 200, 200)) 
    mX, mY = 25,25 
    Bimg.paste(Firstimg.resize(Oimg.size),(mX,mY))  # キャンバスの中央に原画像を配置
    canvas = Bimg.copy()  
    showarray(canvas.resize(np.array(canvas.size)//2))
    canvas.save(fn+"/qimg_00000.png")     

    oldtotalloss = 1e+30
    for step in range(steps):
        progress0.value=step
        pgc = 0
        totalloss = 0
        for h in range(0,height1-223,200):
            for w in range(0,width1-223,200):
                pgc=pgc+1 # プログレスバーのカウンター
                progress1.value = pgc
                
                qimg = canvas.crop((w+mX,h+mY,w+mX+224,h+mY+224))  # Qimgを初期値に使う                
                qimgVGG = VGGprepare(qimg)
                nn.setpic(qimgVGG)
                
                img = Oimg.crop((w,h,w+224,h+224))
                
                rimg,aloss = generate224(nn,img,styleimg,itr=itr,l1=l1, l2=l2, \
                                  dlosslimit=dlosslimit)

                canvas.paste(rimg, (w+mX,h+mY))
                
                totalloss = totalloss + aloss
        
        showarray(canvas.resize(np.array(canvas.size)//2))
        canvas.save(fn+"/all_%05d.png"%step)

        if totalloss > oldtotalloss:
            printf("Total loss が増えすぎているので停止")
            break
        
    return canvas

# 実験用画像

In [ ]:
img = PIL.Image.open("images/momiji.jpg")
img.resize(np.array(img.size)//2)

In [ ]:
img = PIL.Image.open("images/Mucha.png")
img.resize(np.array(img.size)//1)

In [ ]:
img = PIL.Image.open("images/kusama.png")
img.resize(np.array(img.size)//2)

In [ ]:
msurl =  'https://goo.gl/ZCnjHJ'
img = url2img(msurl)
img.resize(np.array(img.size)//3)

In [ ]:
!mkdir ex0
lr = np.float32(10.0) # 学習係数
rr1 = np.float32(0.00)
rr2 = np.float32(0.00)
generate3(nn,"images/momiji.jpg","images/Mucha.png",steps=2,itr=3, fn="ex0")

In [ ]:
!mkdir ex0
lr = np.float32(10.0) # 学習係数
rr1 = np.float32(0.01)
rr2 = np.float32(0.02)
generate3(nn,"images/momiji.jpg","images/Mucha.png",steps=2,itr=3, fn="ex0")

# まず基本
参照原画に似せる項のみ。当然ながらただちに原画そのものになる。ボケてるのは、ぼかして使っているから。

# Musha

In [ ]:
!mkdir ex8
lr = np.float32(10.0) # 学習係数
rr1 = np.float32(0.01)
rr2 = np.float32(0.05)
rr3 = np.float32(0.0)
img = generate3(nn,"images/momiji.jpg","images/Mucha.png",steps=5,itr=5, fn="ex8")

In [ ]:
!mkdir ex8
lr = np.float32(10.0) # 学習係数
rr1 = np.float32(0.01)
rr2 = np.float32(0.05)
rr3 = np.float32(0.0)
img = generate3(nn,"images/momiji.jpg","images/Mucha.png",steps=5,itr=5, fn="ex8")

In [ ]:
!mkdir ex8
lr = np.float32(1.0) # 学習係数
rr1 = np.float32(0.01)
rr2 = np.float32(0.05)
rr3 = np.float32(0.0)
img = generate3(nn,"images/momiji.jpg","images/kusama.png",steps=5,itr=5, fn="ex10")

In [ ]:
!mkdir ex8
lr = np.float32(10.0) # 学習係数
rr1 = np.float32(0.01)
rr2 = np.float32(0.05)
rr3 = np.float32(0.5)
img = generate3(nn,"images/momiji.jpg","images/kusama.png",steps=5,itr=5, fn="ex10")

In [ ]:
!mkdir ex12
lr = np.float32(3.0) # 学習係数
rr1 = np.float32(0.01)
rr2 = np.float32(0.05)
rr3 = np.float32(0.5)
img = generate3(nn,"images/momiji.jpg","images/mstyle.png",steps=20,itr=5, fn="ex12")

In [ ]:
!mkdir ex13
lr = np.float32(1.0) # 学習係数
rr1 = np.float32(0.05)
rr2 = np.float32(0.15)
rr3 = np.float32(0.2)
img = generate3(nn,"images/momiji.jpg","images/mstyle.png",steps=20,itr=5, fn="ex13")

In [ ]:
!mkdir ex12
lr = np.float32(2000.0) # 学習係数
rr1 = np.float32(0.02)
rr2 = np.float32(0.03)
generate(nn,"images/himawari.png",msurl,crop=3,itr=50, fn="ex12")

In [ ]:
!convert -delay 12 -loop 0 -resize 300x tes/*.png tes.gif

つなぎ目が目立たないようにするために，単純に224ｘ224で区切るのではなく，隣接するブロック同士に重なりのエリアを設け，重なり部分は先行して処理したブロックの処理結果で固定化することにする．

### 実験

重なりを24ピクセルとして試してみる．


In [ ]:
!mkdir ex4
lr = np.float32(10.0) # 学習係数
rr1 = np.float32(0.00)
rr2 = np.float32(1)
generate(nn,"images/himawari.png",msurl,itr=30, fn="ex4", interval=30)

In [ ]:
!mkdir ex12
lr = np.float32(10.0) # 学習係数
rr1 = np.float32(0.01)
rr2 = np.float32(0.05)
generate2(nn,"images/Quebec.jpg","images/Mucha.png",tlsteps=5,itr=5, fn="ex12",interval=5)

In [ ]:
# !mkdir ex13
lr = np.float32(10.0) # 学習係数
rr1 = np.float32(0.01)
rr2 = np.float32(0.05)
rr3 = np.float32(0.01)
generate2(nn,"images/Quebec.jpg","images/Mucha.png",tlsteps=5,itr=5, fn="ex13")

In [ ]:
!mkdir ex14
lr = np.float32(10.0) # 学習係数
rr1 = np.float32(0.01)
rr2 = np.float32(0.05)
rr3 = np.float32(0.01)
generate2(nn,"images/Quebec.jpg","images/Mucha.png",tlsteps=5,itr=5, fn="ex14")

In [ ]:
!mkdir ex16
lr = np.float32(10.0) # 学習係数
rr1 = np.float32(0.01)
rr2 = np.float32(0.05)
rr3 = np.float32(0.5)
generate2(nn,"images/Quebec.jpg","images/Mucha.png",tlsteps=5,itr=5, fn="ex16")

In [ ]:
h,steps=1,0
topkeep=True if h > 0 and steps == 0 else False

In [ ]:
topkeep